In [61]:
import jieba
import os
import re

"""
1、读取数据集
2、jieba分词并去停用词
"""
file_path_list = []
root_path = './dataset'
r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:：;<=>?「」@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
stopwords.append('说道')
for file in os.listdir(root_path):
    file_path_list.append(os.path.join(root_path, file))
print(file_path_list)
corpus=[[] for _ in range(16)]
for i,file_path in enumerate(file_path_list):
    with open(file_path,encoding="ANSI") as f:
        text = f.read()[40:]
        text = re.sub(r1, '', text)
        text = text.replace("\n", '')
        text = text.replace("\u3000", '')
        text = text.replace(" ", '')
        jieba_text = jieba.lcut(text)
        for word in jieba_text:
            if word not in stopwords and len(word) > 1:
                corpus[i].append(word)


['./dataset\\三十三剑客图.txt', './dataset\\书剑恩仇录.txt', './dataset\\侠客行.txt', './dataset\\倚天屠龙记.txt', './dataset\\天龙八部.txt', './dataset\\射雕英雄传.txt', './dataset\\白马啸西风.txt', './dataset\\碧血剑.txt', './dataset\\神雕侠侣.txt', './dataset\\笑傲江湖.txt', './dataset\\越女剑.txt', './dataset\\连城诀.txt', './dataset\\雪山飞狐.txt', './dataset\\飞狐外传.txt', './dataset\\鸳鸯刀.txt', './dataset\\鹿鼎记.txt']


In [144]:
"""
1、抽取段落并打标签
"""
num = 1000 // 16
words = 500
num_topics = 50
train = []
label = []
for i in range(len(corpus)): 
    stride = len(corpus[i]) // num
    for j in range (num):
        train.append(corpus[i][stride*j:stride*j + 500])
        label.append(i)

    

In [145]:
"""
1、LDA模型训练
"""
from gensim import corpora, models

dictionary = corpora.Dictionary(train)
lda_corpus_train = [dictionary.doc2bow(tmp_doc) for tmp_doc in train]
print("Trainng LDA model...")
lda = models.LdaModel(corpus=lda_corpus_train, id2word=dictionary, num_topics=num_topics)

Trainng LDA model...


In [146]:
"""
1、文本分类模型训练与测试
"""
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

train_topic_distribution = lda.get_document_topics(lda_corpus_train)
train_features = np.zeros((len(train), num_topics))
for i in range(len(train_topic_distribution)):
    tmp_topic_distribution = train_topic_distribution[i]
    for j in range(len(tmp_topic_distribution)):
        train_features[i][tmp_topic_distribution[j][0]] = tmp_topic_distribution[j][1]



train_data, test_data, train_label, test_label = train_test_split(train_features, label, test_size=0.2, shuffle=True)


print("Training SVM classifier...")
assert len(label) == len(train_features)
train_label = np.array(train_label)
classifier = SVC(kernel='linear', probability=True)
classifier.fit(train_data, train_label)
print("Prediction accuracy of training samples is {:.4f}.".format(sum(classifier.predict(train_data) == train_label) / len(train_label)))

a = sum(classifier.predict(test_data) == test_label) / len(test_label)
print(a)

Training SVM classifier...
Prediction accuracy of training samples is 0.3972.
0.33668341708542715


(39, 50)